In [36]:
import pandas as pd
import numpy as npi

In [37]:
df = pd.read_csv("data_no_spaces_all_features_correct_dates.csv")

In [38]:
df.columns

Index(['Unnamed: 0', 'channelDescription', 'channelJoinedDate',
       'channelTotalVideos', 'channelTotalViews', 'channelUsername',
       'commentsCount', 'date', 'duration', 'id', 'isChannelVerified', 'likes',
       'numberOfSubscribers', 'order', 'text', 'title', 'url', 'viewCount',
       'likes_per_subscriber', 'comments_per_subscriber',
       'views_per_subscriber', 'duration_in_seconds', 'datetime_date',
       'hashtags', 'comm_to_views', 'likes_to_views', 'popular_brand',
       'has_title_affiliate', 'has_description_affiliate',
       'has_channel_description_affiliate', 'has_any_affiliate',
       'has_business_inquiry', 'engagement_per_subscriber'],
      dtype='object')

In [39]:
#df2 = df[["channelUsername","date","title","likes","viewCount", "numberOfSubscribers", "likes_per_subscriber", "views_per_subscriber", "duration_in_seconds"]]

df2 = df 

In [40]:
#df2["days_since_2024_01_01"] = (pd.to_datetime(df2["date"]) - pd.to_datetime("2024-01-01").tz_localize("UTC")).dt.days

In [41]:
#df2 = df2.query("days_since_2024_01_01 <= 366")

In [42]:
#Here I was trying to see if there were any titles that were repeated. 
#What I noticed was that certain content creators have their own theme/running series of videos that they label with specific titles.
#AtarahMayhew uses "I 👅 U" as the title for any video where she swallows a piece of paper after doing her makeup
#AtarahMayhew has a series of "CUTE ✅ or FAIL? ❌ Tiktok Filters Pick My Makeup 😱"
#Julia.P uses "Dior lip maximizer", "Kiko Milano lip gloss", 'New Dior lipstick', 'Fenty beauty lip oil','Queen Musia lipstick'! 
# where shes just showing the packaging with viral transition sounds or songs playing over the audio
#alot of swatches, but sort of low-effort, with no description etc
#nats_tutorials seems to have a running series of "testing a weird ____" which results in several videos of the same title
#lifebyjessica1 has a running series of "dollar store finds"
#lindseyteaches has a running series of teacher hairstyles of the day
#SofiaSeesBeauty was spamming her new brush set/using the title as an advertisement 
#blueyoobin has a 5 step skincare series

#the main offenders for title repetition (they have more than one series running) are: AtarahMayhew, Julia.P, nats_tutorials

counts = df2['title'].value_counts()
with pd.option_context("display.max_rows", None):
    display(counts)


title
Take off my makeup with me! 🫧                                                                           10
ASMR Makeup Tutorial! ✨GRWM💄                                                                             7
5-step skincare                                                                                          7
CUTE ✅ or FAIL? ❌ Tiktok Filters Pick My Makeup 😱                                                        7
Dior single eyeshadow                                                                                    6
Dior lip maximizer                                                                                       6
I 👅 U                                                                                                    6
Kiko Milano lip gloss                                                                                    5
Sofia Sees Beauty x ESUM is here! Shop my 6 piece brush set 15% off w/ code SOFIA at ESUM Cosmetics      5
NYX slick click fat oil        

In [43]:
#I was wondering if there were keywords I wasn't thinking of - just from some bias I have, so instead I search for all words used in every title
#And order them by frequency. 
#I then tried to remove the obvious articles and conjunctions, but somes still come through, maybe some blank space or smthn subtle? 

word_list = []
list_to_ignore = ["for","the","a","me","my","in","for","but","of","this","that","The","with","is","you","&","your","it","do","be","by","so","What","what","With","all","i","I","if","Why","why"]
list_to_ignore = set([word.lower() for word in list_to_ignore])
for _, row in df.iterrows():
    new_words = row["title"].split()
    word_list = word_list + [word.lower() for word in new_words if word not in list_to_ignore]

with pd.option_context("display.max_rows", None):
    display(pd.Series(word_list).value_counts())

#makeup                                             1823
to                                                  1474
makeup                                              1444
#shorts                                             1433
#skincare                                           1228
lip                                                 1069
#beauty                                              876
skin                                                 794
the                                                  792
new                                                  755
i                                                    747
part                                                 729
#kbeauty                                             716
skincare                                             692
how                                                  679
beauty                                               624
on                                                   557
these                          

In [44]:
#streamlining the "presence of a substring" function Rachael was implementing earlier - Jo

def compare_means(df, thing_to_check_for, thing_to_take_average_of, where_to_look="title"):
    print(f"Comparing means of {thing_to_take_average_of} with and without {thing_to_check_for}.")
    idxs = (df[where_to_look].str.count(thing_to_check_for) > 0)
    print(df.loc[idxs, thing_to_take_average_of].mean())
    print(df.loc[~idxs, thing_to_take_average_of].mean())

def get_mean_with_word(df, thing_to_check_for, thing_to_take_average_of, where_to_look="title"):
    idxs = (df[where_to_look].str.count(thing_to_check_for) > 0)
    return df.loc[idxs, thing_to_take_average_of].mean()

def get_mean_without_word(df, thing_to_check_for, thing_to_take_average_of, where_to_look="title"):
    idxs = (df[where_to_look].str.count(thing_to_check_for) == 0)
    return df.loc[idxs, thing_to_take_average_of].mean()


In [45]:
# 12. Identify popular video topics/formats and see if videos that cover these topics perform better than average. 
# For example, one popular video format is "speed reviews."
#"speed reviews", "haul", "dupe" , "GRWM"  etc


# Create a dataframe
schema = pd.DataFrame()

words =["dupe","cheap","drugstore"]# add more words
column_to_take_average_of = "likes_per_subscriber"
where_to_look_for_word = "title"

#added color vs colour to see if there was an american bias
#try vs tryon, since try includes both tryon, as well as "trying" 
#celebrity names to see if people want to copy signature looks?
#asmr, grwm, vlog have secondary content separate from the objective makeup 
#retail stores to see if people are looking to purchase
#ten - trying to see if people like lists? i.e. "my top ten"

schema["word"] = words


# Adds column, containing means of likes_per_subscriber of entries with the substring
schema['with_string'] = schema["word"].apply(lambda word : get_mean_with_word(df, word, column_to_take_average_of, where_to_look_for_word))

# Adds column, containing means of likes_per_subscriber of entries without the substring
schema['without_string'] = schema["word"].apply(lambda word : get_mean_without_word(df, word, column_to_take_average_of, where_to_look_for_word))


# Adds column, containing means of likes_per_subscriber of entries without the substring
schema['difference'] = schema['with_string']-df["likes_per_subscriber"].mean()


#finding diffrence between with and without substring

schema = schema.sort_values('difference', ascending=False)

# show dataframe 

print(df["likes_per_subscriber"].mean())
display(schema)

0.03903806369560744


,word,with_string,without_string,difference
2,drugstore,0.097120,0.038739,0.058082
1,cheap,0.075682,0.039031,0.036644
0,dupe,0.055142,0.038977,0.016104


In [46]:
#keyword groups
comparison_list = ["unbox","haul","review","try","vs"]
skills_list=["tip","trick","hack","tutorial"]
skincare_list = ["skincare","mask","toner"]#"sunscreen","acne","clean","snail","serum"
product_list=["tint","lipgloss","blush","balm","foundation","lipstick","concealer","eyeshadow","mascara","oil"] #
speed_list=["short","speed","quick","fast","routine"]
#adjective_list=["color","colour","shade","swatch"]
#brand_noun_list=["fenty","dior"]
budget_list=["dupe","cheap","drugstore"] #"budget","affordable"
self_ref_list=["viral","short","popular","fav","best","cute","easy","trend","makeup","beauty"]
acronym_list=["grwm","ootd","asmr"]
kniche_list=["kbeauty","korean"]
#texture_list=["jelly","gel","matte","glitter"]
#season=["summer","winter","fall","valentine","spring"]

In [47]:
def contains_substring(text, substring_list):
    for substring in substring_list:
        if substring.lower() in text.lower():
            return True
    return False

def contains_products(text):
    return 1 * contains_substring(text, product_list)
def contains_budget(text):
    return 1 * contains_substring(text, budget_list)
def contains_self_ref(text):
    return 1 * contains_substring(text, self_ref_list)
def contains_acronym(text):
    return 1 * contains_substring(text, acronym_list)
def contains_kniche(text):
    return 1 * contains_substring(text, kniche_list)
def contains_speed(text):
    return 1 * contains_substring(text, speed_list)
def contains_skills(text):
    return 1 * contains_substring(text, skills_list)
def contains_comparison(text):
    return 1 * contains_substring(text, comparison_list)
def contains_skincare(text):
    return 1 * contains_substring(text, skincare_list)


#skincare and acronym are somewhat poorly performing, but I left them in for now, and we can decide?
#the three groups I would recommend are "comparing_products","skills/teach","speed" and "self_ref". 
#korean is there because it just does very well, would be hard to ignore its effect but doesnt really fit into a group?
#budget is there because...it feels intuitive to mention, just does moderately well

df2 = df2.copy()

df2["product"] = df2["title"].apply(contains_products)
df2["budget"] = df2["title"].apply(contains_budget)
df2["self_ref"] = df2["title"].apply(contains_self_ref)
df2["acronym"] = df2["title"].apply(contains_acronym)
df2["korean"] = df2["title"].apply(contains_kniche)
df2["speed"] = df2["title"].apply(contains_speed)
df2["skills/teach"] = df2["title"].apply(contains_skills)
df2["skincare"] = df2["title"].apply(contains_skincare)
df2["comparing_products"] = df2["title"].apply(contains_comparison)

In [48]:
display(df2)

,Unnamed: 0,channelDescription,channelJoinedDate,channelTotalVideos,channelTotalViews,channelUsername,commentsCount,date,duration,id,...,engagement_per_subscriber,product,budget,self_ref,acronym,korean,speed,skills/teach,skincare,comparing_products
0,0,Business inquiries only: info@nikkietutorials....,"Jun 23, 2008",909.0,2.092121e+09,nikkietutorials,1844.0,2025-01-01T17:03:25.000Z,00:01:20,IF6k0pRUGVM,...,0.056438,1,0,0,0,0,0,0,0,0
1,1,Business inquiries only: info@nikkietutorials....,"Jun 23, 2008",909.0,2.092121e+09,nikkietutorials,1845.0,2024-09-26T17:09:34.000Z,00:00:59,qmrnz6Ee3Vs,...,0.032055,0,0,1,0,0,0,0,0,1
2,2,Business inquiries only: info@nikkietutorials....,"Jun 23, 2008",909.0,2.092121e+09,nikkietutorials,1398.0,2024-12-13T17:06:39.000Z,00:01:06,iMhxKAQmHuc,...,0.022534,0,0,1,0,0,0,0,0,1
3,3,Business inquiries only: info@nikkietutorials....,"Jun 23, 2008",909.0,2.092121e+09,nikkietutorials,529.0,2025-02-16T18:19:19.000Z,00:01:25,bM3XWjv8lfI,...,0.008219,0,0,1,0,0,0,0,0,1
4,4,Business inquiries only: info@nikkietutorials....,"Jun 23, 2008",909.0,2.092121e+09,nikkietutorials,4539.0,2024-07-07T17:33:45.000Z,00:01:00,L0mCS-xdzbc,...,0.045342,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15041,17499,Hi!! My name is Thivi and I love everything be...,"Jul 28, 2021",385.0,4.855238e+06,Lordthivii,13.0,2024-12-03T21:02:47.000Z,00:00:55,o0Bdcmf76tk,...,0.210657,1,0,1,0,0,0,0,0,1
15042,17500,Hi!! My name is Thivi and I love everything be...,"Jul 28, 2021",385.0,4.855238e+06,Lordthivii,3.0,2024-12-03T21:13:12.000Z,00:00:40,072nA9RKncQ,...,0.005452,1,0,1,0,1,0,0,0,1
15043,17501,Hi!! My name is Thivi and I love everything be...,"Jul 28, 2021",385.0,4.855238e+06,Lordthivii,14.0,2024-12-03T20:57:23.000Z,00:00:34,fZpT7ynLPwM,...,0.210657,1,0,1,0,0,0,0,0,1
15044,17513,Hi!! My name is Thivi and I love everything be...,"Jul 28, 2021",385.0,4.855238e+06,Lordthivii,3.0,2024-07-31T00:01:49.000Z,00:00:39,A4poIRObwcs,...,0.032218,0,1,0,0,0,0,0,0,0


In [49]:
df2["budget"].value_counts()

budget
0    14535
1      511
Name: count, dtype: int64

In [50]:
df2["product"].value_counts()

product
0    12156
1     2890
Name: count, dtype: int64

In [51]:
df2["self_ref"].value_counts()

self_ref
0    7653
1    7393
Name: count, dtype: int64

In [52]:
df2["acronym"].value_counts()

acronym
0    14520
1      526
Name: count, dtype: int64

In [53]:
df2["comparing_products"].value_counts()

comparing_products
0    13551
1     1495
Name: count, dtype: int64

In [54]:
df2["skills/teach"].value_counts()

skills/teach
0    13544
1     1502
Name: count, dtype: int64

In [55]:
df2["speed"].value_counts()

speed
0    12659
1     2387
Name: count, dtype: int64

In [56]:
df2["korean"].value_counts()

korean
0    13987
1     1059
Name: count, dtype: int64

In [57]:
keywords = ["product", "skills/teach", "speed", "comparing_products", "self_ref", "budget", "korean"]

for keyword in keywords:
    print(df2.groupby(keyword)["viewCount"].mean())

product
0    359813.067703
1    319354.168858
Name: viewCount, dtype: float64
skills/teach
0    356412.867174
1    312626.715047
Name: viewCount, dtype: float64
speed
0    368531.653843
1    264591.115626
Name: viewCount, dtype: float64
comparing_products
0    360260.833961
1    277542.901672
Name: viewCount, dtype: float64
self_ref
0    406318.056710
1    295856.771405
Name: viewCount, dtype: float64
budget
0    361811.287994
1     74156.806262
Name: viewCount, dtype: float64
korean
0    334784.902338
1    579966.732767
Name: viewCount, dtype: float64


In [58]:
for keyword in keywords:
    print(df2.groupby(keyword)["likes"].mean())

product
0    20195.268263
1    16629.236678
Name: likes, dtype: float64
skills/teach
0    19984.873892
1    15231.053928
Name: likes, dtype: float64
speed
0    20731.925745
1    13031.724759
Name: likes, dtype: float64
comparing_products
0    19813.946425
1    16758.118395
Name: likes, dtype: float64
self_ref
0    23292.815889
1    15594.786284
Name: likes, dtype: float64
budget
0    20027.826144
1     4790.062622
Name: likes, dtype: float64
korean
0    18350.892758
1    34823.643059
Name: likes, dtype: float64


In [59]:
for keyword in keywords:
    print(df2.groupby(keyword)["likes_per_subscriber"].mean())

product
0    0.032625
1    0.066012
Name: likes_per_subscriber, dtype: float64
skills/teach
0    0.039442
1    0.035394
Name: likes_per_subscriber, dtype: float64
speed
0    0.032579
1    0.073291
Name: likes_per_subscriber, dtype: float64
comparing_products
0    0.037426
1    0.053647
Name: likes_per_subscriber, dtype: float64
self_ref
0    0.026112
1    0.052418
Name: likes_per_subscriber, dtype: float64
budget
0    0.038997
1    0.040204
Name: likes_per_subscriber, dtype: float64
korean
0    0.031912
1    0.133157
Name: likes_per_subscriber, dtype: float64


In [60]:
for keyword in keywords:
    print(df2.groupby(keyword)["views_per_subscriber"].mean())

product
0    0.695602
1    1.644879
Name: views_per_subscriber, dtype: float64
skills/teach
0    0.873518
1    0.917783
Name: views_per_subscriber, dtype: float64
speed
0    0.736247
1    1.629362
Name: views_per_subscriber, dtype: float64
comparing_products
0    0.836328
1    1.255088
Name: views_per_subscriber, dtype: float64
self_ref
0    0.634023
1    1.130429
Name: views_per_subscriber, dtype: float64
budget
0    0.861593
1    1.342824
Name: views_per_subscriber, dtype: float64
korean
0    0.725641
1    2.889427
Name: views_per_subscriber, dtype: float64


In [70]:
df2.to_csv("correct_dates_with_keywords.csv")